In [1]:
import sys
import numpy as np
import tensorflow as tf
from tensorflow import keras

if '..' not in sys.path:
    sys.path.append('..')
from dlml.nn import compute_receptive_field

tf.random.set_seed(5061983)

### Build a simple model

This CNN will be used to compute effective receptive field size and effective stride. Note that using an activation function or a max pooling layer (instead of the average pooling used below) might invalidate the first assertion in the last cell. To understand why, consider for instance the case in which we are using a max pooling layer: when we change the value of a sample inside the receptive field, this change will not be propagated to the downstream layer if the changed value is not the maximum in the pooling window. As a consequence, the assertion will fail, even though the computed effective receptive field size is correct.

In [2]:
N_samples = 4096
filters = [2, 4, 8]
kernel_size = [4, 8, 16]
kernel_stride = [1, 2, 4]
pool_size = [2, 4, 8]
pool_stride = [2, 4, 8]
fc_units = [16, 1]
kernel_init = 'glorot_uniform'
use_bias = False
bias_init = 'ones' if use_bias else 'zeros'
input_layer = keras.layers.Input(shape=(N_samples,1), name='input', dtype=tf.float32)
L = input_layer
for i,(n,ksz,kstrd,psz,pstrd) in enumerate(zip(filters, kernel_size, kernel_stride,
                                               pool_size, pool_stride)):
    L = keras.layers.Conv1D(filters=n,
                            kernel_size=ksz,
                            strides=kstrd,
                            activation=None,
                            padding='valid',
                            kernel_initializer=kernel_init,
                            use_bias=use_bias,
                            bias_initializer=bias_init,
                            name=f'conv_{i+1}')(L)
#     L = keras.layers.ReLU(name=f'relu_{i+1}')(L)
    if psz is not None:
        L = keras.layers.AveragePooling1D(pool_size=psz,
                                          strides=pstrd,
                                          name=f'avg_pool_{i+1}')(L)
#         L = keras.layers.MaxPooling1D(pool_size=psz, name=f'max_pool_{i+1}')(L)
L = keras.layers.Flatten(name='flatten')(L)
for i,n in enumerate(fc_units):
    L = keras.layers.Dense(units=n,
                           activation='ReLU',
                           kernel_initializer='ones',
                           name=f'fc_{i+1}')(L)
model = keras.Model(inputs=[input_layer], outputs=[L])
multi_output_model = keras.Model(inputs=[model.layers[0].input],
                                 outputs=[layer.output for layer in model.layers[1:]])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 4096, 1)]         0         
                                                                 
 conv_1 (Conv1D)             (None, 4093, 2)           8         
                                                                 
 avg_pool_1 (AveragePooling1  (None, 2046, 2)          0         
 D)                                                              
                                                                 
 conv_2 (Conv1D)             (None, 1020, 4)           64        
                                                                 
 avg_pool_2 (AveragePooling1  (None, 255, 4)           0         
 D)                                                              
                                                                 
 conv_3 (Conv1D)             (None, 60, 8)             512   

2022-05-23 11:45:49.693901: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Compute effective receptive field size and effective stride for each layer in the model

We stop at the `Flatten` layer: RF size and stride make no sense for fully connected layers

In [3]:
effective_RF_size,effective_stride = compute_receptive_field(model, stop_layer='flatten')
print('Effective receptive field size:')
for i,(k,v) in enumerate(effective_RF_size.items()):
    print(f'{i}. {k} ' + '.' * (20 - len(k)) + ' {:d}'.format(v))
print()
print('Effective stride:')
for i,(k,v) in enumerate(effective_stride.items()):
    print(f'{i}. {k} ' + '.' * (20 - len(k)) + ' {:d}'.format(v))

Effective receptive field size:
0. input ............... 1
1. conv_1 .............. 4
2. avg_pool_1 .......... 5
3. conv_2 .............. 19
4. avg_pool_2 .......... 31
5. conv_3 .............. 271
6. avg_pool_3 .......... 719

Effective stride:
0. input ............... 1
1. conv_1 .............. 1
2. avg_pool_1 .......... 2
3. conv_2 .............. 4
4. avg_pool_2 .......... 16
5. conv_3 .............. 64
6. avg_pool_3 .......... 512


### Test that the computed values are correct

To do so, we change one sample inside and one outside the effective receptive field: we expect that the output will change only when the modified sample is inside the RF.

In [4]:
layer_name = 'avg_pool_3'
for layer_id,layer in enumerate(model.layers):
    if layer.name == layer_name:
        break
_, N_neurons, N_kernels = model.layers[layer_id].output.shape
neuron = np.random.randint(0, N_neurons)
kernel = np.random.randint(0, N_kernels)
x = np.random.uniform(size=(1, N_samples))
x_mod_1 = x.copy()
x_mod_2 = x.copy()

# change the last value inside the receptive field
x_mod_1[0, effective_stride[layer_name] * neuron + effective_RF_size[layer_name] - 1] *= 2

# change the first value outside the receptive field
x_mod_2[0, effective_stride[layer_name] * neuron + effective_RF_size[layer_name]] *= 2

multi_y = multi_output_model(x)
multi_y_mod_1 = multi_output_model(x_mod_1)
multi_y_mod_2 = multi_output_model(x_mod_2)
assert multi_y[layer_id-1][0, neuron, kernel] != multi_y_mod_1[layer_id-1][0, neuron, kernel]
assert multi_y[layer_id-1][0, neuron, kernel] == multi_y_mod_2[layer_id-1][0, neuron, kernel]